### Tips from prof

- Narrow scope of work (e.g. court level)

- Could try both binary/multi-class model outcomes and compare the performance 

- Change user from layperson to legal professional (and mention that this project is a stepping stone towards having layperson use the model)

- Link features to predicted outcome (if time permits can try using XGBoost with LIME for model interpretability)

- Can also try to see accuracy of models with different areas of law, lowest accuracy may be hardest area of law to predict


### Data setup

In [208]:
import re
from spacy import displacy
import json
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel, CoherenceModel, LsiModel, HdpModel
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
import ast
import nltk
import pyLDAvis
import pyLDAvis.gensim_models

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\benhz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\benhz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [209]:
# Load CSV files into DataFrames
areas_of_law_df = pd.read_csv("data/prediction_data/areas_of_law.csv")
coram_df = pd.read_csv("data/prediction_data/coram.csv")
sg_legal_cases_df = pd.read_csv("data/prediction_data/sg_legal_cases_dataset.csv")
target_rulings_df = pd.read_csv("data/prediction_data/target_rulings.csv")
issues_facts_df = pd.read_csv("data/prediction_data/issues_facts_topic.csv")
# Load the JSON file into a dictionary
with open('data/prediction_data/issues.json') as f:
    issues_data = [json.loads(line) for line in f]
issues_df = pd.DataFrame(issues_data)

# Load the JSON file into a dictionary
with open('data/rawish_data/facts.json') as f:
    facts_data = [json.loads(line) for line in f]
raw_facts_df = pd.DataFrame(facts_data)

# Merge DataFrames
merged_df = pd.merge(areas_of_law_df, sg_legal_cases_df, on='casename', how='outer')
merged_df = pd.merge(merged_df, issues_df, on='casename', how='outer')
merged_df = pd.merge(merged_df, raw_facts_df, on='casename', how='outer')
merged_df = pd.merge(merged_df, issues_facts_df, on='casename', how='outer')
merged_df = pd.merge(merged_df, target_rulings_df, on='casename', how='outer')
merged_df = merged_df.reset_index(drop=True)
try:
    merged_df.drop(columns=['Unnamed: 0'], inplace=True)
except:
    pass
# Display the resulting DataFrame
print(merged_df.head())

           casename                                        area_of_law  \
0   2000_SGCA_1.pdf  {'civil procedure': ['pleadings'], 'res judica...   
1  2000_SGCA_10.pdf  {'contract': ['formation'], 'equity': ['defenc...   
2  2000_SGCA_11.pdf  {'contract': ['discharge'], 'damages': ['asses...   
3  2000_SGCA_12.pdf  {'courts and jurisdiction': ['court of appeal'...   
4  2000_SGCA_13.pdf                     {'criminal law': ['offences']}   

  court_level                                             issues  \
0        SGCA  The claim was dismissed with costs by the\nHig...   
1        SGCA  the claim and\nagainst that decision this appe...   
2        SGCA  The appeal \nThe questions which arise in this...   
3        SGCA  the appeals from the assistant registrar. In h...   
4        SGCA  the appeal on 24 January 2000 and dismissed it...   

                                               facts  issues_topic  \
0  The facts\nThe appellant is the widow of one T...          12.0   
1  fac

### Data Preprocessing

Remove duplicate coram names and roles

In [210]:
def clean_coram_names(coram_list):
    all_names = set()
    for item in coram_list:
        split_names = re.split(r';\s(?![a-zA-Z]+\s)', item)
        for name in split_names:
            if ';' in name and not re.search(r';\s[a-zA-Z]+$', name):
                sub_names = name.split(';')
                all_names.update([n.strip() for n in sub_names if n.strip()])
            else:
                all_names.add(name.strip())
    return list(all_names)

def remove_coram_roles(coram_list):
    roles = [' CJ', ' AG', ' J', ' DCJ', ' JA', ' AR', ' JC', 'SAR']
    for role in roles:
        coram_list = [re.sub(rf'{role}$', '', name) for name in coram_list]
    return coram_list

In [211]:
coram_df = coram_df.dropna()
for i, coram_str in enumerate(coram_df['Coram']):
    coram = ast.literal_eval(coram_str)
    
    coram_modified = clean_coram_names(coram)
    coram_modified = remove_coram_roles(coram_modified)
    coram_df.at[i, 'Coram'] = str(coram_modified)
merged_df = pd.merge(merged_df, coram_df, on='casename', how='outer')

In [212]:
nan_counts = merged_df.isna().sum()
print(nan_counts)

#nas are probably those reassigned cases, coram has 7, i just drop them for now
na_target_rows = merged_df[merged_df['target'].isna()]
print(na_target_rows)

merged_df.dropna(axis=0, inplace=True)
print(merged_df.isna().sum())

#remove empty lists
merged_df = merged_df.query("area_of_law != '[]'")

#target is unbalanced
target_counts = merged_df['target'].value_counts()
print(target_counts)

merged_df = merged_df.reset_index(drop=True) # prevent nan values from appearing after one-hot

casename         7
area_of_law      7
court_level      7
issues          54
facts           54
issues_topic    54
facts_topic     54
target          54
Unnamed: 0      14
Coram            7
dtype: int64
               casename area_of_law court_level issues facts  issues_topic  \
241   2000_SGHC_257.pdf          []        SGHC    NaN   NaN           NaN   
274   2000_SGHC_290.pdf          []        SGHC    NaN   NaN           NaN   
412    2001_SGCA_66.pdf          []        SGCA    NaN   NaN           NaN   
432   2001_SGHC_101.pdf          []        SGHC    NaN   NaN           NaN   
438   2001_SGHC_108.pdf          []        SGHC    NaN   NaN           NaN   
442   2001_SGHC_111.pdf          []        SGHC    NaN   NaN           NaN   
448   2001_SGHC_118.pdf          []        SGHC    NaN   NaN           NaN   
457   2001_SGHC_128.pdf          []        SGHC    NaN   NaN           NaN   
460   2001_SGHC_130.pdf          []        SGHC    NaN   NaN           NaN   
462   2001_SGHC_1

In [213]:
merged_df['area_of_law'] = merged_df['area_of_law'].apply(ast.literal_eval)
merged_df['Coram'] = merged_df['Coram'].apply(ast.literal_eval)
merged_df.head(3)

,casename,area_of_law,court_level,issues,facts,issues_topic,facts_topic,target,Unnamed: 0,Coram
0,2000_SGCA_1.pdf,"{'civil procedure': ['pleadings'], 'res judica...",SGCA,The claim was dismissed with costs by the\nHig...,The facts\nThe appellant is the widow of one T...,12.0,7.0,Favourable,4258.0,"[V K Rajah, Chan Sek Keong, Andrew Phang Boon ..."
1,2000_SGCA_10.pdf,"{'contract': ['formation'], 'equity': ['defenc...",SGCA,the claim and\nagainst that decision this appe...,facts and surrounding circumstances including ...,8.0,3.0,Favourable,7628.0,"[V K Rajah, Chao Hick Tin, Andrew Phang Boon L..."
2,2000_SGCA_11.pdf,"{'contract': ['discharge'], 'damages': ['asses...",SGCA,The appeal \nThe questions which arise in this...,"Background \nThe first appellants, a French co...",0.0,12.0,No outcome,7876.0,"[Tan Lee Meng, Chan Sek Keong, Andrew Phang Bo..."


##### Flatten areas_of_law

Using Flattened AOL

In [214]:
def clean_list(law_list):
    cleaned_list = [item.replace(']', '').strip() for item in law_list]  # Removes ']' and extra spaces
    return cleaned_list

In [215]:
def flatten_keys_only(d):
    if isinstance(d, dict):
        return list(d.keys())
    return [] 

In [216]:
def flatten_areas(areas):
    flat_areas = []
    for main_area, sub_areas in areas.items():
        flat_areas.append(main_area)
        # Remove sub-area if it's longer than 33 characters
        sub_areas = [sarea for sarea in sub_areas if len(sarea) <= 33]
        flat_areas.extend(sub_areas)
    return flat_areas

In [217]:
# merged_df['area_of_law'] = merged_df['area_of_law'].apply(flatten_keys_only)
merged_df['area_of_law'] = merged_df['area_of_law'].apply(flatten_areas)
merged_df['area_of_law'] = merged_df['area_of_law'].apply(clean_list)

In [218]:
merged_df

,casename,area_of_law,court_level,issues,facts,issues_topic,facts_topic,target,Unnamed: 0,Coram
0,2000_SGCA_1.pdf,"[civil procedure, res judicata, trusts]",SGCA,The claim was dismissed with costs by the\nHig...,The facts\nThe appellant is the widow of one T...,12.0,7.0,Favourable,4258.0,"[V K Rajah, Chan Sek Keong, Andrew Phang Boon ..."
1,2000_SGCA_10.pdf,"[contract, equity]",SGCA,the claim and\nagainst that decision this appe...,facts and surrounding circumstances including ...,8.0,3.0,Favourable,7628.0,"[V K Rajah, Chao Hick Tin, Andrew Phang Boon L..."
2,2000_SGCA_11.pdf,"[contract, damages]",SGCA,The appeal \nThe questions which arise in this...,"Background \nThe first appellants, a French co...",0.0,12.0,No outcome,7876.0,"[Tan Lee Meng, Chan Sek Keong, Andrew Phang Bo..."
3,2000_SGCA_12.pdf,"[courts and jurisdiction, words and phrases]",SGCA,the appeals from the assistant registrar. In h...,"Background\nMicrosoft, Adobe and Autodesk are ...",27.0,10.0,Unfavourable,8424.0,[Teo Guan Siew]
4,2000_SGCA_13.pdf,[criminal law],SGCA,the appeal on 24 January 2000 and dismissed it...,facts. Mere assertion would not suffice. In ex...,28.0,13.0,Unfavourable,8158.0,[Lionel Yee]
...,...,...,...,...,...,...,...,...,...,...
6786,2023_SGHC_95.pdf,[criminal law],SGHC,the issues (and sub-issue) that arise for my \...,"Facts \n1 The first accused, Low Sze Song (“Lo...",15.0,11.0,No outcome,4512.0,[Chan Seng Onn]
6787,2023_SGHC_96.pdf,[tort],SGHC,"the claim for conspiracy to defraud). At most,...","the facts, and can only be giving his opinion,...",10.0,1.0,Unfavourable,5054.0,"[Tay Yong Kwang, Judith Prakash, Sundaresh Men..."
6788,2023_SGHC_97.pdf,[civil procedure],SGHC,the issues relating to the defences of \njusti...,"Facts\nThe parties \n3 The plaintiff, Mr Karan...",18.0,10.0,Unfavourable,5062.0,[Woo Bih Li]
6789,2023_SGHC_98.pdf,[insolvency law],SGHC,,,0.0,0.0,Favourable,3995.0,[Judith Prakash]


Using main area ONLY

One-hot Encoding

In [219]:
# one-hot encode aol
processed_df = merged_df[merged_df['area_of_law'].apply(lambda x: isinstance(x, list))]
processed_df = processed_df.reset_index(drop=True)
mlb = MultiLabelBinarizer()
binary_features = mlb.fit_transform(processed_df['area_of_law'])
binary_aol_df = pd.DataFrame(binary_features, columns=mlb.classes_)
binary_aol_df = binary_aol_df.reset_index(drop=True)
processed_df = pd.concat([processed_df.drop('area_of_law', axis=1), binary_aol_df], axis=1)

print(processed_df.head(3))

           casename court_level  \
0   2000_SGCA_1.pdf        SGCA   
1  2000_SGCA_10.pdf        SGCA   
2  2000_SGCA_11.pdf        SGCA   

                                              issues  \
0  The claim was dismissed with costs by the\nHig...   
1  the claim and\nagainst that decision this appe...   
2  The appeal \nThe questions which arise in this...   

                                               facts  issues_topic  \
0  The facts\nThe appellant is the widow of one T...          12.0   
1  facts and surrounding circumstances including ...           8.0   
2  Background \nThe first appellants, a French co...           0.0   

   facts_topic      target  Unnamed: 0  \
0          7.0  Favourable      4258.0   
1          3.0  Favourable      7628.0   
2         12.0  No outcome      7876.0   

                                               Coram  \
0  [V K Rajah, Chan Sek Keong, Andrew Phang Boon ...   
1  [V K Rajah, Chao Hick Tin, Andrew Phang Boon L...   
2  [Tan Lee Meng

In [220]:
# one-hot encode coram
processed_df = processed_df[processed_df['Coram'].apply(lambda x: isinstance(x, list))]

mlb = MultiLabelBinarizer()
binary_features = mlb.fit_transform(processed_df['Coram'])
binary_coram_df = pd.DataFrame(binary_features, columns=mlb.classes_)
binary_coram_df = binary_coram_df.reset_index(drop=True)
processed_df = pd.concat([processed_df.drop('Coram', axis=1), binary_coram_df], axis=1)

print(processed_df.head())

           casename court_level  \
0   2000_SGCA_1.pdf        SGCA   
1  2000_SGCA_10.pdf        SGCA   
2  2000_SGCA_11.pdf        SGCA   
3  2000_SGCA_12.pdf        SGCA   
4  2000_SGCA_13.pdf        SGCA   

                                              issues  \
0  The claim was dismissed with costs by the\nHig...   
1  the claim and\nagainst that decision this appe...   
2  The appeal \nThe questions which arise in this...   
3  the appeals from the assistant registrar. In h...   
4  the appeal on 24 January 2000 and dismissed it...   

                                               facts  issues_topic  \
0  The facts\nThe appellant is the widow of one T...          12.0   
1  facts and surrounding circumstances including ...           8.0   
2  Background \nThe first appellants, a French co...           0.0   
3  Background\nMicrosoft, Adobe and Autodesk are ...          27.0   
4  facts. Mere assertion would not suffice. In ex...          28.0   

   facts_topic        target  U

In [221]:
processed_df['SGCA'] = processed_df['court_level'].apply(lambda x: 1 if x == 'SGCA' else 0)
processed_df['SGHC'] = processed_df['court_level'].apply(lambda x: 1 if x == 'SGHC' else 0)
processed_df = processed_df.drop('court_level', axis=1)

### Convolutional Neural Networks (CNN)

Train & Test set for CNN

In [222]:
X = processed_df.drop(columns=['target'])
y = processed_df['target']

stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
for train_index, remaining_index in stratified_split.split(X, y):
    X_train, X_test_val = X.iloc[train_index], X.iloc[remaining_index]
    y_train, y_test_val = y.iloc[train_index], y.iloc[remaining_index]

#balanced dataset (target variable was imbalanced Favourable 5006 Unfavourable 2523 No outcome 984)
#randomly found one online, can be changed -> need to check am i doing this right 
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

#split further from X_test_val into X_val and X_test
X_val, X_test, y_val, y_test = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=42, stratify=y_test_val)

Convert target variable to continuous

In [223]:
X_train_resampled = X_train_resampled.drop(columns=['facts', 'issues'])
X_test = X_test.drop(columns=['facts', 'issues'])
X_val = X_val.drop(columns=['facts', 'issues'])

mapping = {'Favourable': 1, 'Unfavourable': 0, 'No outcome':0.5}

y_train_resampled, y_test, y_val = y_train_resampled.copy().map(mapping), y_test.copy().map(mapping), y_val.copy().map(mapping)

#### Modeling (CNN)

In [224]:
# Perform modelling
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import tensorflow as tf

Hyper params

In [225]:
class Args:
  epochs = 20
  lr = 0.001
  use_cuda=False
  gamma = 0.7
  log_interval = 10
  seed = 1

args = Args()

device = torch.device("cuda" if args.use_cuda else "cpu")

In [226]:
X_train_resampled = X_train_resampled.iloc[:, 1:1596].copy()
X_train_resampled = torch.tensor(X_train_resampled.values, dtype=torch.float32).to(device)
print(f'Shape of X_train_resampled: {X_train_resampled.shape}')

X_test = X_test.iloc[:, 1:1596].copy()
X_test = torch.tensor(X_test.values, dtype=torch.float32).to(device)
print(f'Shape of X_test: {X_test.shape}')

X_val = X_val.iloc[:, 1:1596].copy()
X_val = torch.tensor(X_val.values, dtype=torch.float32).to(device)
print(f'Shape of X_val: {X_val.shape}')

Shape of X_train_resampled: torch.Size([8274, 197])
Shape of X_test: torch.Size([1019, 197])
Shape of X_val: torch.Size([1019, 197])


In [227]:
y_train_resampled, y_test, y_val = torch.tensor(y_train_resampled.values).to(device), torch.tensor(y_test.values).to(device), torch.tensor(y_val.values).to(device)

X_train_resampled = X_train_resampled.reshape(X_train_resampled.shape[0],1,X_train_resampled.shape[1])
X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])
X_val = X_val.reshape(X_val.shape[0],1,X_val.shape[1])
print(X_train_resampled.shape)

torch.Size([8274, 1, 197])


#### Training CNN

In [228]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(1, 64, 3, 1,1, bias=True)
        # Define the first 1D convolution layer. Takes 1 input channel, outputs 32 channels, kernel size is 3, stride is 1, padding is 1.
        self.Bn1 = nn.BatchNorm1d(64)
        # Apply Batch Normalization to the output of the first convolutional layer.
        self.dropout = nn.Dropout(0.3)
        self.pool1 = nn.AvgPool1d(kernel_size=2, stride=2)
        # Apply 1D Average Pooling after the first Batch Normalization. The kernel size and stride are 2.

        self.conv2 = nn.Conv1d(64, 64, 3, 1,1, bias=True)
        self.Bn2 = nn.BatchNorm1d(64)
        self.pool2 = nn.AvgPool1d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(3136, 100, bias=True)
        # Define the first fully connected layer. It takes 25472 inputs and outputs 100 nodes.

        self.fc2 = nn.Linear(100, 30, bias=True)
        # Define the second fully connected layer. It takes 100 inputs and outputs 50 nodes.

        self.fc3 = nn.Linear(30, 3, bias=True)
        # Define the third fully connected layer (output layer). It takes 50 inputs and outputs 3 nodes.


    def forward(self, x):
        x = F.relu(self.Bn1(self.conv1(x)))
        # Pass the input through the first convolutional layer, then Batch Normalization, and then apply ReLU activation.
        x = self.dropout(x)
        x = self.pool1(x)
        # Apply Average Pooling to the output of the previous step.
        x = F.relu(self.Bn2(self.conv2(x)))
        x = self.dropout(x)
        x = self.pool2(x)
        x = torch.flatten(x, 1)
        # Flatten the output from the previous step. This is necessary because fully connected layers expect a 1D input.
        x = self.fc1(x)
        # Pass the output through the first fully connected layer.
        x = F.relu(self.fc2(x))
        # Pass the output through the second fully connected layer with ReLU activation.
        x = self.fc3(x)
        # Pass the output through the third fully connected layer. This is the output of the network.
        return x
    
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()  # Set the model to training mode

    for batch_idx, (data, target) in enumerate(train_loader):  # Loop over each batch from the training set
        data, target = data.to(device), target.to(device)  # Move the data to the device that is used

        target = target.long()  # Make sure that target data is long type (necessary for loss function)

        optimizer.zero_grad()  # Clear gradients from the previous training step
        output = model(data)  # Run forward pass (model predictions)

        loss = F.cross_entropy(output, target)  # Calculate the loss between the output and target
        loss.backward()  # Perform backpropagation (calculate gradients of loss w.r.t. parameters)
        optimizer.step()  # Update the model parameters

        if batch_idx % args.log_interval == 0:  # Print log info for specified interval
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset),100. * batch_idx / len(train_loader), loss.item()))



def test(model, device, test_loader):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0
    correct = 0

    with torch.no_grad():  # Deactivates autograd, reduces memory usage and speeds up computations
        for data, target in test_loader:  # Loop over each batch from the testing set
            
            data, target = data.to(device), target.to(device)  # Move the data to the device that is used

            target = target.long()  # Convert target to long after adjusting value
            output = model(data)  # Run forward pass (model predictions)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # Sum up the batch loss
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability as the predicted output
            correct += pred.eq(target.view_as(pred)).sum().item()  # Count correct predictions

    test_loss /= len(test_loader.dataset)  # Calculate the average loss

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset),100. * correct / len(test_loader.dataset)))
    return correct  # Return the number of correctly classified samples


In [229]:
torch.manual_seed(args.seed)

model = Net().to(device)

for param_tensor in model.state_dict():
        print(param_tensor, "\t", model.state_dict()[param_tensor].size())

#Form training and testing dataset
optimizer = optim.Adam(model.parameters(), lr=args.lr)

train_dataset = torch.utils.data.TensorDataset(X_train_resampled, y_train_resampled)
test_dataset = torch.utils.data.TensorDataset(X_val, y_val)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)
scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)

#Model training
ACC = 0
for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    ACC_ = test(model, device, test_loader)
    if ACC_>ACC or ACC_ == ACC:
        ACC = ACC_
        torch.save(model.state_dict(), "Baseline_CNN.pt")

    scheduler.step()

print(ACC)


conv1.weight 	 torch.Size([64, 1, 3])
conv1.bias 	 torch.Size([64])
Bn1.weight 	 torch.Size([64])
Bn1.bias 	 torch.Size([64])
Bn1.running_mean 	 torch.Size([64])
Bn1.running_var 	 torch.Size([64])
Bn1.num_batches_tracked 	 torch.Size([])
conv2.weight 	 torch.Size([64, 64, 3])
conv2.bias 	 torch.Size([64])
Bn2.weight 	 torch.Size([64])
Bn2.bias 	 torch.Size([64])
Bn2.running_mean 	 torch.Size([64])
Bn2.running_var 	 torch.Size([64])
Bn2.num_batches_tracked 	 torch.Size([])
fc1.weight 	 torch.Size([100, 3136])
fc1.bias 	 torch.Size([100])
fc2.weight 	 torch.Size([30, 100])
fc2.bias 	 torch.Size([30])
fc3.weight 	 torch.Size([3, 30])
fc3.bias 	 torch.Size([3])
Train Epoch: 1 [0/8274 (0%)]	Loss: 1.075337
Train Epoch: 1 [640/8274 (8%)]	Loss: 0.645248


Train Epoch: 1 [1280/8274 (15%)]	Loss: 0.701008
Train Epoch: 1 [1920/8274 (23%)]	Loss: 0.673608
Train Epoch: 1 [2560/8274 (31%)]	Loss: 0.664185
Train Epoch: 1 [3200/8274 (38%)]	Loss: 0.657935
Train Epoch: 1 [3840/8274 (46%)]	Loss: 0.696572
Train Epoch: 1 [4480/8274 (54%)]	Loss: 0.603941
Train Epoch: 1 [5120/8274 (62%)]	Loss: 0.613066
Train Epoch: 1 [5760/8274 (69%)]	Loss: 0.664815
Train Epoch: 1 [6400/8274 (77%)]	Loss: 0.602779
Train Epoch: 1 [7040/8274 (85%)]	Loss: 0.720407
Train Epoch: 1 [7680/8274 (92%)]	Loss: 0.674416

Test set: Average loss: 0.8144, Accuracy: 428/1019 (42%)

Train Epoch: 2 [0/8274 (0%)]	Loss: 0.595437
Train Epoch: 2 [640/8274 (8%)]	Loss: 0.643984
Train Epoch: 2 [1280/8274 (15%)]	Loss: 0.636484
Train Epoch: 2 [1920/8274 (23%)]	Loss: 0.714236
Train Epoch: 2 [2560/8274 (31%)]	Loss: 0.689526
Train Epoch: 2 [3200/8274 (38%)]	Loss: 0.631262
Train Epoch: 2 [3840/8274 (46%)]	Loss: 0.665805
Train Epoch: 2 [4480/8274 (54%)]	Loss: 0.635389
Train Epoch: 2 [5120/8274 (62%)]	Lo

### Random Forest (RF)

In [230]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV

#### Train & Test set for RF

In [231]:
X = processed_df.drop(['target', 'casename'], axis=1)
y = processed_df['target'] 

# Handle imbalanced classes
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

# Splitting the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

In [232]:
# Vectorizing textual features using TF-IDF for X_train
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_text = tfidf_vectorizer.fit_transform(X_train['facts'].astype('U') + ' ' + X_train['issues'].astype('U'))
X_train_text = pd.DataFrame(X_train_text.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Drop original text columns and concatenate TF-IDF features
X_train = X_train.drop(['facts', 'issues'], axis=1)
X_train = pd.concat([X_train.reset_index(drop=True), X_train_text], axis=1)

# Vectorizing textual features using TF-IDF for X_test
X_test_text = tfidf_vectorizer.transform(X_test['facts'].astype('U') + ' ' + X_test['issues'].astype('U'))
X_test_text = pd.DataFrame(X_test_text.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Drop original text columns and concatenate TF-IDF features
X_test = X_test.drop(['facts', 'issues'], axis=1)
X_test = pd.concat([X_test.reset_index(drop=True), X_test_text], axis=1)


#### Modelling (RF)

Hyper Param Tuning (Grid search)

In [233]:
rf_classifier = RandomForestClassifier(random_state=42)
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Setup the grid search
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)

# Fit grid search
grid_search.fit(X_train, y_train)

# Best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

# Optionally, use the best estimator to make predictions
best_rf = grid_search.best_estimator_

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best cross-validation score: 0.83


Model Training

In [234]:
# Train the model using the training sets
best_rf.fit(X_train, y_train)

y_pred = best_rf.predict(X_test)

# Evaluating the Model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.8522695235410206
Classification Report:
              precision    recall  f1-score   support

  Favourable       0.74      0.86      0.80      1195
  No outcome       0.98      0.98      0.98      1165
Unfavourable       0.85      0.72      0.78      1187

    accuracy                           0.85      3547
   macro avg       0.86      0.85      0.85      3547
weighted avg       0.86      0.85      0.85      3547



### Evaluation